* Seeting up a virtual environment is needed, the codes are in the following:


conda create -n econml-py39 python=3.9
conda activate econml-py39
pip install numpy==1.23.5 scipy==1.10.1 pandas==1.5.3 scikit-learn==1.2.2 matplotlib==3.6.3 seaborn==0.12.2 xgboost==1.7.5 lightgbm==3.3.5 shap==0.43.0 econml==0.15.0

pip install ipykernel
python -m ipykernel install --user --name econml-py39 --display-name econml-py39


In [1]:
from econml.grf import CausalForest, CausalIVForest, RegressionForest
import numpy as np
import scipy.special
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from linearmodels.panel import PanelOLS

Goal of casual models: Estimating \tao in a observational dataset (not a randomlized)

In [3]:
df = pd.read_csv('cemlcbq2vszshspq.csv')
print(df.columns.tolist())


['costat', 'curcd', 'datafmt', 'indfmt', 'consol', 'tic', 'fyear', 'gvkey', 'conm', 'cusip', 'exchg', 'fic', 'county', 'gind', 'sic', 'curuscn', 'aocidergl', 'at', 'lt', 'cidergl', 'ni', 'capx', 'derac', 'deralt', 'derhedgl', 'derlc', 'derllt', 'dt', 'mkvalt']


C:\Users\85832\AppData\Local\Temp\ipykernel_64128\155378091.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('cemlcbq2vszshspq.csv')


costat: Company Status
curcd: Currency Code
datafmt: Data Format
indfmt: Industry Format
tic: Ticker Symbol
fyear: Fiscal Year
gvkey: Global Company Key（used to identify companies since there're missing tic according to the table below）
conm: Company Name
cusip: CUSIP Identifier
exchg: Exchange Code
fic: Foreign Incorporation Code
county: Country Code
gind: Global Industry Code
sic: Standard Industry Classification Code, can used to seperate industry affect(dummy). It is divided into 10 catagories by its first digit.
curuscn: Current US Currency Conversion Rate X
aocidergl: Accumulated Other Comprehensive Income - Derivative Unrealized Gains/Losses
at: Assets - Total
lt: Liabilities - Total
cidergl: Comprehensive Income - Derivative Gains/Losses
ni: Net Income
capx: Capital Expenditures
derac: Derivative Assets - Current
deralt: Derivative Assets - Long-Term
derhedgl: Derivative Hedging Gain/Loss
derlc: Derivative Liabilities - Current
derllt: Derivative Liabilities - Long-Term
dt: Total debt including current
mkvalt: Market Value - Total

In [10]:
total = len(df)
non_na_count = df['derac'].notna().sum()
non_na_ratio = non_na_count / total #the ratio included in the 
print(non_na_ratio)
df_non_na = df['derac'].sum

0.1022755172644689


In [14]:
#Building needed variables for this research:
#Treatment: Tobins'Q 

print(total)
df.isna().sum()

179212


costat            0
curcd             0
datafmt           0
indfmt            0
consol            0
tic             122
fyear             0
gvkey             0
conm              0
cusip           112
exchg           112
fic               0
county       179202
gind          42798
sic               0
curuscn      158252
aocidergl     65349
at            44790
lt            44960
cidergl       62368
ni            61088
capx          61974
derac        160883
deralt       159973
derhedgl     155520
derlc        156146
derllt       155836
dt            73355
mkvalt        72298
tobins_q      72298
dtype: int64

In [64]:
# Filtering data by total assets,total debt, market value. If there was any 0 value during the research period, the company(gvkey) is delated
to_exclude = df[(df['at'] == 0) | (df['dt'] == 0) | (df['mkvalt'] == 0)
                  ].gvkey.unique()
filtered_data = df[~df['gvkey'].isin(to_exclude)]
filtered_data = filtered_data.dropna(subset= ['dt','mkvalt','capx','at','ni','lt',])
filtered_data.isna().sum()
filtered_len = len(filtered_data)
filtered_data['tobins_q'] = (filtered_data['mkvalt']) / filtered_data['at']
filtered_data['ROA'] = filtered_data['ni']/filtered_data['at']
filtered_data['der_ratio'] = filtered_data['derhedgl']/filtered_data['at']
filtered_data['derhedgl_binary'] = filtered_data['derhedgl'].apply(lambda x: 0 if pd.isna(x) else 1) #control derivative use as binary variable
#filtered_data['der']
filtered_data['total_der'] = filtered_data[]


#Creating dummy SIC 10 groups (main industry)

filtered_data['sic_group'] = filtered_data['sic'].astype(str).str[:1]  # First digit

# creating dummies for different industries 
sic_group_dummies = pd.get_dummies(filtered_data['sic_group'], prefix='sic_group')
filtered_data = pd.concat([filtered_data, sic_group_dummies], axis=1)
print(filtered_data)








       costat curcd datafmt indfmt consol     tic  fyear   gvkey  \
0           A   USD     STD   INDL      C     AIR   2010    1004   
1           I   USD     STD   INDL      C  ADCT.1   2010    1013   
2           I   USD     STD   INDL      C    AFAP   2010    1019   
3           A   USD     STD   INDL      C     AAL   2010    1045   
9           A   USD     STD   INDL      C     PRG   2010    1076   
...       ...   ...     ...    ...    ...     ...    ...     ...   
179187      A   USD     STD   INDL      C    NYAX   2023  348615   
179190      A   USD     STD   INDL      C    MDAI   2023  349337   
179193      A   USD     STD   INDL      C    INDP   2023  349972   
179198      A   USD     STD   INDL      C   IVCGF   2023  351491   
179200      A   USD     STD   INDL      C    CLCO   2023  352262   

                               conm      cusip  ...  sic_group sic_group_1  \
0                          AAR CORP  000361105  ...          5           0   
1        ADC TELECOMMUNICAT

In [ ]:
print(filtered_data.describe())

gvkey   fyear
1004    2010      True
1013    2010      True
1019    2010      True
1045    2010      True
1076    2010      True
                 ...  
348615  2023      True
349337  2023     False
349972  2023      True
351491  2023     False
352262  2023      True
Name: derhedgl, Length: 41662, dtype: bool


In [65]:
print(filtered_len) #sample size after filtered
filtered_data.to_csv('filtered_data.csv', index=False)

41662


In [66]:
#sort filtered data
filtered_data = filtered_data.set_index(['gvkey','fyear'])
filtered_data.head()

,,costat,curcd,datafmt,indfmt,consol,tic,conm,cusip,exchg,fic,...,sic_group,sic_group_1,sic_group_2,sic_group_3,sic_group_4,sic_group_5,sic_group_6,sic_group_7,sic_group_8,sic_group_9
gvkey,fyear,,,,,,,,,,,,,,,,,,,,,
1004,2010,A,USD,STD,INDL,C,AIR,AAR CORP,000361105,11.0,USA,...,5,0,0,0,0,1,0,0,0,0
1013,2010,I,USD,STD,INDL,C,ADCT.1,ADC TELECOMMUNICATIONS INC,000886309,14.0,USA,...,3,0,0,1,0,0,0,0,0,0
1019,2010,I,USD,STD,INDL,C,AFAP,AFA PROTECTIVE SYSTEMS INC,001038108,19.0,USA,...,7,0,0,0,0,0,0,1,0,0
1045,2010,A,USD,STD,INDL,C,AAL,AMERICAN AIRLINES GROUP INC,02376R102,14.0,USA,...,4,0,0,0,1,0,0,0,0,0
1076,2010,A,USD,STD,INDL,C,PRG,PROG HOLDINGS INC,74319R101,11.0,USA,...,6,0,0,0,0,0,1,0,0,0


The following part is to investigate how is the relation between Tobins'q or Market value and controal variables and derivative related variables.

Panel regressions with fixed effects were used here, different financial indicators were created above, and some more variables can be added such as leverage ratio.

In [77]:
base_exog = ['derac','deralt','at','lt','ni','ROA','der_ratio','derhedgl_binary']
#sic_cols = [col for col in filtered_data.columns if str(col).startswith('sic_group_')]
#sic_cols = sic_cols[:-1] 
exog_vars = base_exog #+sic_cols
exog = filtered_data[exog_vars]
reg_fe = PanelOLS(filtered_data.mkvalt, exog, entity_effects= True, drop_absorbed=True)
results_fe = reg_fe.fit()
print(results_fe)


                          PanelOLS Estimation Summary                           
Dep. Variable:                 mkvalt   R-squared:                        0.5628
Estimator:                   PanelOLS   R-squared (Between):             -0.1005
No. Observations:                4486   R-squared (Within):               0.5628
Date:                Sat, Oct 18 2025   R-squared (Overall):              0.4753
Time:                        15:46:38   Log-likelihood                -5.186e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      638.36
Entities:                        1008   P-value                           0.0000
Avg Obs:                       4.4504   Distribution:                  F(7,3471)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             638.36
                            

C:\Users\85832\anaconda3\envs\econml-py39\lib\site-packages\linearmodels\panel\model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [80]:
base_exog = ['at','lt','ni','ROA','derhedgl_binary']
#sic_cols = [col for col in filtered_data.columns if str(col).startswith('sic_group_')]
#sic_cols = sic_cols[:-1] 
exog_vars = base_exog #+sic_cols
exog = filtered_data[exog_vars]
reg_fe = PanelOLS(filtered_data.mkvalt, exog, entity_effects= True, drop_absorbed=True)
results_fe = reg_fe.fit()
print(results_fe)


                          PanelOLS Estimation Summary                           
Dep. Variable:                 mkvalt   R-squared:                        0.3419
Estimator:                   PanelOLS   R-squared (Between):              0.4970
No. Observations:               41662   R-squared (Within):               0.3419
Date:                Sat, Oct 18 2025   R-squared (Overall):              0.5453
Time:                        15:48:59   Log-likelihood                -4.675e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3699.5
Entities:                        6052   P-value                           0.0000
Avg Obs:                       6.8840   Distribution:                 F(5,35605)
Min Obs:                       1.0000                                           
Max Obs:                       14.000   F-statistic (robust):             3699.5
                            

In [84]:
base_exog = ['at','lt','ni','ROA','derhedgl_binary']
#sic_cols = [col for col in filtered_data.columns if str(col).startswith('sic_group_')]
#sic_cols = sic_cols[:-1] 
exog_vars = base_exog #+sic_cols
exog = filtered_data[exog_vars]
reg_fe = PanelOLS(filtered_data.tobins_q, exog, entity_effects= True, drop_absorbed=True)
results_fe = reg_fe.fit()
print(results_fe)


                          PanelOLS Estimation Summary                           
Dep. Variable:               tobins_q   R-squared:                        0.0002
Estimator:                   PanelOLS   R-squared (Between):             -0.0018
No. Observations:               41662   R-squared (Within):               0.0002
Date:                Sat, Oct 18 2025   R-squared (Overall):             -0.0010
Time:                        15:50:02   Log-likelihood                 -3.29e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1.3121
Entities:                        6052   P-value                           0.2555
Avg Obs:                       6.8840   Distribution:                 F(5,35605)
Min Obs:                       1.0000                                           
Max Obs:                       14.000   F-statistic (robust):             1.3121
                            

In [94]:
filtered_data['L.der_ratio'] = filtered_data.groupby('gvkey')['der_ratio'].shift(1)
filtered_data['L.derac'] = filtered_data.groupby('gvkey')['derac'].shift(1)
filtered_data.head()

base_exog = ['derac','lt','ni','ROA','L.der_ratio']
sic_cols = [col for col in filtered_data.columns if str(col).startswith('sic_group_')]
sic_cols = sic_cols[:-1] 
exog_vars = base_exog +sic_cols
exog = filtered_data[exog_vars]
reg_fe = PanelOLS(filtered_data.tobins_q, exog, entity_effects= True,time_effects=True, drop_absorbed=True)
results_fe = reg_fe.fit()
print(results_fe)
# This regression shows industry dummies were abosred by entity effects



                          PanelOLS Estimation Summary                           
Dep. Variable:               tobins_q   R-squared:                        0.7401
Estimator:                   PanelOLS   R-squared (Between):              0.6193
No. Observations:                4904   R-squared (Within):               0.7402
Date:                Sat, Oct 18 2025   R-squared (Overall):              0.6961
Time:                        16:36:19   Log-likelihood                -2.834e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2151.3
Entities:                        1113   P-value                           0.0000
Avg Obs:                       4.4061   Distribution:                  F(5,3777)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             2151.3
                            

C:\Users\85832\anaconda3\envs\econml-py39\lib\site-packages\linearmodels\panel\model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
C:\Users\85832\AppData\Local\Temp\ipykernel_10804\2859951839.py:11: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

sic_group_1, sic_group_2, sic_group_3, sic_group_4, sic_group_5, sic_group_6, sic_group_7, sic_group_8

  results_fe = reg_fe.fit()


In [101]:
#Choosing firms that are top10% on market value in order to control firm value effects
avg_mkt = filtered_data.groupby('gvkey')['mkvalt'].mean().reset_index()
threshold = avg_mkt['mkvalt'].quantile(0.9)
top10_gvkeys = avg_mkt.loc[avg_mkt['mkvalt'] >= threshold, 'gvkey'].tolist()

#
top10p = filtered_data[filtered_data.index.get_level_values('gvkey').isin(top10_gvkeys)].copy()


In [107]:

# reg for top 10%, lagged derivative ratio(hedge/total assets is significant)
exog_vars = ['derac','at','lt','ni','ROA','L.der_ratio']  
X = top10p[exog_vars]
y = top10p['tobins_q']

reg = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)
res = reg.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:               tobins_q   R-squared:                        0.0746
Estimator:                   PanelOLS   R-squared (Between):              0.0775
No. Observations:                1404   R-squared (Within):               0.0746
Date:                Sat, Oct 18 2025   R-squared (Overall):              0.0861
Time:                        17:18:28   Log-likelihood                   -1379.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      15.656
Entities:                         233   P-value                           0.0000
Avg Obs:                       6.0258   Distribution:                  F(6,1165)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             15.656
                            

C:\Users\85832\anaconda3\envs\econml-py39\lib\site-packages\linearmodels\panel\model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


['gvkey', 'fyear']
